# Imports

In [59]:
import glob
import os
import shutil
import re

import pandas as pd
import scanpy as sc
import sig_reorg
import sig_utils
from joblib import Parallel, delayed
from pandas_utils import describe
from tqdm import tqdm

import create_sourmash_command_utils as csc_utils

%load_ext autoreload
%autoreload 2

pd.options.display.max_rows = 500

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read one2one h5ad

In [2]:
adata = sc.read(
#     "/home/phoenix/data_lg/data_sm_copy/code/immune-evolution/h5ads/human-lemur-mouse-bat__lung_only_copy.h5ad"
    "/home/olga/data_lg/data_sm_copy/immune-evolution/h5ads/human-lemur-mouse-bat/human-lemur-mouse-bat__lung_only.h5ad"
)
adata.obs = adata.obs.reset_index().set_index('cell_id')
print(adata)
adata.obs.head()

AnnData object with n_obs × n_vars = 126745 × 10560
    obs: 'index', 'age', 'cell_barcode', 'cell_ontology_class', 'cell_ontology_id', 'channel', 'free_annotation', 'individual', 'sample', 'sequencing_run', 'sex', 'species', 'species_batch', 'species_latin', 'tissue', 'narrow_group', 'broad_group', 'compartment_group', 'compartment_narrow', 'channel_cleaned', 'batch', 'n_genes', 'n_counts', 'species_batch_v2', 'compartment_broad', 'compartment_broad_narrow', 'compartment_species', 'compartment_narrow_species', 'common_individual_id'
    var: 'bat__gene_name', 'mouse_lemur__gene_name-bat', 'mouse__gene_name-bat', 'mouse_lemur__gene_name_x-hlm', 'mouse__gene_name_x-hlm', 'gene_ids-lemur-hlm', 'n_cells-mouse-hlm', 'mouse_lemur__gene_name_y-hlm', 'mouse__gene_name_y-hlm'


,index,age,cell_barcode,cell_ontology_class,cell_ontology_id,channel,free_annotation,individual,sample,sequencing_run,...,channel_cleaned,batch,n_genes,n_counts,species_batch_v2,compartment_broad,compartment_broad_narrow,compartment_species,compartment_narrow_species,common_individual_id
cell_id,,,,,,,,,,,,,,,,,,,,,
P2_1__AAACCTGAGAAACCAT,P2_1_AAACCTGAGAAACCAT-human-hlm,46y,AAACCTGAGAAACCAT,capillary endothelial cell,nan,P2_1,Capillary Aerocyte,patient_2,distal 2,180607_A00111_0155_BHFCWYDMXX,...,P2_1,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,human_patient_2
P2_1__AAATGCCAGATGAGAG,P2_1_AAATGCCAGATGAGAG-human-hlm,46y,AAATGCCAGATGAGAG,capillary endothelial cell,nan,P2_1,Capillary Aerocyte,patient_2,distal 2,180607_A00111_0155_BHFCWYDMXX,...,P2_1,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,human_patient_2
P2_1__AACACGTTCGATCCCT,P2_1_AACACGTTCGATCCCT-human-hlm,46y,AACACGTTCGATCCCT,capillary endothelial cell,nan,P2_1,Capillary Aerocyte,patient_2,distal 2,180607_A00111_0155_BHFCWYDMXX,...,P2_1,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,human_patient_2
P2_1__AACACGTTCGCACTCT,P2_1_AACACGTTCGCACTCT-human-hlm,46y,AACACGTTCGCACTCT,capillary endothelial cell,nan,P2_1,Capillary Aerocyte,patient_2,distal 2,180607_A00111_0155_BHFCWYDMXX,...,P2_1,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,human_patient_2
P2_1__AACCATGCAGCTCGCA,P2_1_AACCATGCAGCTCGCA-human-hlm,46y,AACCATGCAGCTCGCA,capillary endothelial cell,nan,P2_1,Capillary Aerocyte,patient_2,distal 2,180607_A00111_0155_BHFCWYDMXX,...,P2_1,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,human_patient_2


In [3]:
adata_shared = adata[adata.obs.broad_group.isin(sig_reorg.SHARED_CELLTYPES)]
adata_shared

/home/olga/miniconda3/envs/immune-evolution/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


View of AnnData object with n_obs × n_vars = 89795 × 10560
    obs: 'index', 'age', 'cell_barcode', 'cell_ontology_class', 'cell_ontology_id', 'channel', 'free_annotation', 'individual', 'sample', 'sequencing_run', 'sex', 'species', 'species_batch', 'species_latin', 'tissue', 'narrow_group', 'broad_group', 'compartment_group', 'compartment_narrow', 'channel_cleaned', 'batch', 'n_genes', 'n_counts', 'species_batch_v2', 'compartment_broad', 'compartment_broad_narrow', 'compartment_species', 'compartment_narrow_species', 'common_individual_id'
    var: 'bat__gene_name', 'mouse_lemur__gene_name-bat', 'mouse__gene_name-bat', 'mouse_lemur__gene_name_x-hlm', 'mouse__gene_name_x-hlm', 'gene_ids-lemur-hlm', 'n_cells-mouse-hlm', 'mouse_lemur__gene_name_y-hlm', 'mouse__gene_name_y-hlm'

# Mouse2mouse comparison

## Input folders

In [4]:
bat_kmermaid_base = '/home/olga/data_lg/data_sm_copy/immune-evolution/pipeline-results/bat/kmermaid/ksize_8'
mouse_kmermaid_base = '/home/olga/data_lg/data_sm_copy/immune-evolution/pipeline-results/mouse/kmermaid/lung--mouse--remove-ribo/'

mouse_peptide_sketch_dir = os.path.join(
    bat_kmermaid_base,
    "sketches_peptide_handmade_mouse_merged_remove_ribosomal_dissociation_all_mouse_cells",
)
mouse_nucleotide_sketch_dir = os.path.join(mouse_kmermaid_base, 'sketches_nucleotide_handmade_per_mouse_remove_ribosomal')

## Set output folder names

In [78]:
base_outdir = (
    "/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse"
)

# --- Training data directories --- #
train_dir = os.path.join(base_outdir, "0--train-mouse1")
train_0_fastas = os.path.join(train_dir, "0--protein-coding-fastas")
train_1_single_cells = os.path.join(train_dir, "1--single-cell-sigs")
train_2_merged_celltype = os.path.join(train_dir, "2--merged-celltype-sigs")
train_3_merged_celltype_remove_common = os.path.join(
    train_dir, "3--merged-celltype-remove-common-kmers"
)
train_4_celltype_sbts = os.path.join(train_dir, "4--celltype-sbt-dbts")

# --- Test data directories --- #
test_dir = os.path.join(base_outdir, "1--test-mouse2")
test_0_fastas = os.path.join(test_dir, "0--protein-coding-fastas")
test_1_single_cells = os.path.join(test_dir, "1--single-cell-sigs")

# --- Results/output directories --- #
results_dir = os.path.join(base_outdir, "2--search-results")

dirs = (
    base_outdir,
    train_dir,
    train_0_fastas,
    train_1_single_cells,
    train_2_merged_celltype,
    train_3_merged_celltype_remove_common,
    train_4_celltype_sbts,
    test_dir,
    test_0_fastas,
    test_1_single_cells,
    results_dir,
)

for outdir in dirs:
    if not os.path.exists(outdir):
        os.makedirs(outdir)

## Query adata to only `MACA_18m_M_LUNG_52`

In [6]:
adata.obs.channel.value_counts()

Antoine_Lung_CD31_10X     8239
Antoine_Lung_P3_10X       7913
10X_P1_6                  7728
bat3-LU                   6454
P3_7                      6254
Bernard_Lung_10X          5958
P2_5                      5803
Antoine_Lung_EPCAM_10X    5419
P3_6                      5237
P3_5                      4325
P2_8                      4064
P2_7                      4055
P2_1                      4039
P3_3                      3880
P2_2                      3391
Stumpy_Lung_10X           3345
Martine_Lung_2_10X        3084
Martine_Lung_1_10X        2976
P2_3                      2829
P1_4                      2826
P1_3                      2407
bat2-LU                   2202
P1_2                      2148
P3_2                      1925
10X_P8_13                 1636
MACA_18m_M_LUNG_52        1573
10X_P1_14                 1565
MACA_21m_F_LUNG_55        1526
P3_4                      1457
10X_P5_14                 1384
P2_6                      1381
P2_4                      1332
MACA_21m

In [7]:
mouse_self_channel = "MACA_18m_M_LUNG_52"
self2self_metadata = adata_shared.obs.query('channel == @mouse_self_channel')
describe(self2self_metadata)

(1172, 29)
--- First 5 entries ---


,index,age,cell_barcode,cell_ontology_class,cell_ontology_id,channel,free_annotation,individual,sample,sequencing_run,...,channel_cleaned,batch,n_genes,n_counts,species_batch_v2,compartment_broad,compartment_broad_narrow,compartment_species,compartment_narrow_species,common_individual_id
cell_id,,,,,,,,,,,,,,,,,,,,,
MACA_18m_M_LUNG_52__AAACCTGAGACACTAA,AAACCTGAGACACTAA-1-31-0-0-mouse-hlm,18m,AAACCTGAGACACTAA,bronchial smooth muscle cell,nan,MACA_18m_M_LUNG_52,Capillary,18-M-52,nan,171019_A00111_0077_BH3YKNDMXX,...,MACA_18m_M_LUNG_52,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary,endothelial: mouse,endothelial: Capillary: mouse,mouse_18-M-52
MACA_18m_M_LUNG_52__AAACCTGCAAAGCGGT,AAACCTGCAAAGCGGT-1-31-0-0-mouse-hlm,18m,AAACCTGCAAAGCGGT,B cell,nan,MACA_18m_M_LUNG_52,B,18-M-52,nan,171019_A00111_0077_BH3YKNDMXX,...,MACA_18m_M_LUNG_52,NaN,NaN,NaN,hlm,lymphoid: B cell,lymphoid: B cell: B cell,lymphoid: mouse,lymphoid: B cell: mouse,mouse_18-M-52
MACA_18m_M_LUNG_52__AAACCTGCACGTCTCT,AAACCTGCACGTCTCT-1-31-0-0-mouse-hlm,18m,AAACCTGCACGTCTCT,bronchial smooth muscle cell,nan,MACA_18m_M_LUNG_52,Capillary,18-M-52,nan,171019_A00111_0077_BH3YKNDMXX,...,MACA_18m_M_LUNG_52,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary,endothelial: mouse,endothelial: Capillary: mouse,mouse_18-M-52
MACA_18m_M_LUNG_52__AAACCTGGTGTCCTCT,AAACCTGGTGTCCTCT-1-31-0-0-mouse-hlm,18m,AAACCTGGTGTCCTCT,"CD8-positive, alpha-beta T cell",nan,MACA_18m_M_LUNG_52,CD8+ T,18-M-52,nan,171019_A00111_0077_BH3YKNDMXX,...,MACA_18m_M_LUNG_52,NaN,NaN,NaN,hlm,lymphoid: T cell,lymphoid: T cell: CD8+ T,lymphoid: mouse,lymphoid: CD8+ T: mouse,mouse_18-M-52
MACA_18m_M_LUNG_52__AAACGGGCAAACCTAC,AAACGGGCAAACCTAC-1-31-0-0-mouse-hlm,18m,AAACGGGCAAACCTAC,bronchial smooth muscle cell,nan,MACA_18m_M_LUNG_52,Capillary,18-M-52,nan,171019_A00111_0077_BH3YKNDMXX,...,MACA_18m_M_LUNG_52,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary,endothelial: mouse,endothelial: Capillary: mouse,mouse_18-M-52


# Training data

## Copy mouse1 fastas

In [ ]:
1+1

In [ ]:
mouse1_obs = adata_shared.obs.query('channel == "MACA_18m_M_LUNG_52"')

mouse_fasta_input_dir = os.path.join(mouse_kmermaid_base, "translate")

sig_reorg.copy_fastas(
    train_0_fastas,
    mouse_fasta_input_dir,
    select_cell_ids=mouse1_obs.index,
    dryrun=False,
)

## Copy mouse1 sketches

In [8]:
mouse1_obs = adata_shared.obs.query('channel == "MACA_18m_M_LUNG_52"')

sig_reorg.copy_nucleotide_peptide_sketches(
    peptide_sketch_dir=mouse_peptide_sketch_dir,
    nucleotide_sketch_dir=mouse_nucleotide_sketch_dir,
    dryrun=False,
    pre_sketch_id_outdir=train_1_single_cells,
    select_cell_ids=mouse1_obs.index
)

0it [00:00, ?it/s]

Copying alphabet-DNA__ksize-21__scaled-10


7109it [01:43, 68.98it/s] 
12it [00:00, 96.41it/s]

Copying alphabet-protein__ksize-30__scaled-10


7109it [00:41, 170.46it/s]
0it [00:00, ?it/s]

Copying alphabet-dayhoff__ksize-51__scaled-10


7109it [01:23, 84.77it/s] 


### Read all copied signatures into dataframe

In [13]:

train_sig_df = pd.Series(
    glob.glob(os.path.join(train_1_single_cells, "*", "*.sig")),
    name="sig_path",
).to_frame()
describe(train_sig_df)

# Get iteration, sketch id
df = (
    train_sig_df["sig_path"]
    .str.split(train_1_single_cells + "/")
    .str[-1]
    .str.split("/", expand=True)
)
describe(df)


sketch_info_df = df[0].str.extractall(sig_utils.SKETCH_INFO_PATTERN)
sketch_info_df.index = sketch_info_df.index.droplevel(-1)
describe(sketch_info_df)
train_sig_df = pd.concat(
    [train_sig_df, df, sketch_info_df], axis=1, ignore_index=True
)
train_sig_df = train_sig_df.rename(
    columns={
        0: "sig_path",
        1: "sketch_id",
        2: "sig_basename",
        3: "alpha_moltype",
        4: "alphabet",
        5: "ksize",
        6: "sketch_style",
        7: "sketch_value",
    }
)
train_sig_df["cell_id"] = (
    train_sig_df["sig_basename"].str.split(".").str[0]
)

# Sourmash can't handle when ksize is str
train_sig_df['ksize'] = train_sig_df.ksize.astype(int)
describe(train_sig_df)

(3516, 1)
--- First 5 entries ---


,sig_path
0,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...
1,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...
2,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...
3,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...
4,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...


(3516, 2)
--- First 5 entries ---


,0,1
0,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__GCATACATCGAGGTAG.sig
1,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__AGGCCGTTCACTTATC.sig
2,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__CGCGGTACAGCAGTTT.sig
3,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__CGGACACAGAGATGAG.sig
4,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__CCATTCGTCGCTGATA.sig


(3516, 5)
--- First 5 entries ---


,mol_or_alpha,alphabet,ksize,sketch_style,sketch_value
0,alphabet,dayhoff,51,scaled,10
1,alphabet,dayhoff,51,scaled,10
2,alphabet,dayhoff,51,scaled,10
3,alphabet,dayhoff,51,scaled,10
4,alphabet,dayhoff,51,scaled,10


(3516, 9)
--- First 5 entries ---


,sig_path,sketch_id,sig_basename,alpha_moltype,alphabet,ksize,sketch_style,sketch_value,cell_id
0,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__GCATACATCGAGGTAG.sig,alphabet,dayhoff,51,scaled,10,MACA_18m_M_LUNG_52__GCATACATCGAGGTAG
1,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__AGGCCGTTCACTTATC.sig,alphabet,dayhoff,51,scaled,10,MACA_18m_M_LUNG_52__AGGCCGTTCACTTATC
2,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__CGCGGTACAGCAGTTT.sig,alphabet,dayhoff,51,scaled,10,MACA_18m_M_LUNG_52__CGCGGTACAGCAGTTT
3,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__CGGACACAGAGATGAG.sig,alphabet,dayhoff,51,scaled,10,MACA_18m_M_LUNG_52__CGGACACAGAGATGAG
4,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__CCATTCGTCGCTGATA.sig,alphabet,dayhoff,51,scaled,10,MACA_18m_M_LUNG_52__CCATTCGTCGCTGATA


In [15]:
train_sig_df_celltypes = train_sig_df.join(adata_shared.obs, on='cell_id')
describe(train_sig_df_celltypes)

(3516, 38)
--- First 5 entries ---


,sig_path,sketch_id,sig_basename,alpha_moltype,alphabet,ksize,sketch_style,sketch_value,cell_id,index,...,channel_cleaned,batch,n_genes,n_counts,species_batch_v2,compartment_broad,compartment_broad_narrow,compartment_species,compartment_narrow_species,common_individual_id
0,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__GCATACATCGAGGTAG.sig,alphabet,dayhoff,51,scaled,10,MACA_18m_M_LUNG_52__GCATACATCGAGGTAG,GCATACATCGAGGTAG-1-31-0-0-mouse-hlm,...,MACA_18m_M_LUNG_52,NaN,NaN,NaN,hlm,myeloid: Monocyte,myeloid: Monocyte: Monocyte,myeloid: mouse,myeloid: Monocyte: mouse,mouse_18-M-52
1,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__AGGCCGTTCACTTATC.sig,alphabet,dayhoff,51,scaled,10,MACA_18m_M_LUNG_52__AGGCCGTTCACTTATC,AGGCCGTTCACTTATC-1-31-0-0-mouse-hlm,...,MACA_18m_M_LUNG_52,NaN,NaN,NaN,hlm,myeloid: Monocyte,myeloid: Monocyte: Monocyte,myeloid: mouse,myeloid: Monocyte: mouse,mouse_18-M-52
2,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__CGCGGTACAGCAGTTT.sig,alphabet,dayhoff,51,scaled,10,MACA_18m_M_LUNG_52__CGCGGTACAGCAGTTT,CGCGGTACAGCAGTTT-1-31-0-0-mouse-hlm,...,MACA_18m_M_LUNG_52,NaN,NaN,NaN,hlm,lymphoid: T cell,lymphoid: T cell: CD8+ T,lymphoid: mouse,lymphoid: CD8+ T: mouse,mouse_18-M-52
3,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__CGGACACAGAGATGAG.sig,alphabet,dayhoff,51,scaled,10,MACA_18m_M_LUNG_52__CGGACACAGAGATGAG,CGGACACAGAGATGAG-1-31-0-0-mouse-hlm,...,MACA_18m_M_LUNG_52,NaN,NaN,NaN,hlm,epithelial: Alveolar Epithelial Type 2,epithelial: Alveolar Epithelial Type 2: Alveol...,epithelial: mouse,epithelial: Alveolar Epithelial Type 2: mouse,mouse_18-M-52
4,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...,alphabet-dayhoff__ksize-51__scaled-10,MACA_18m_M_LUNG_52__CCATTCGTCGCTGATA.sig,alphabet,dayhoff,51,scaled,10,MACA_18m_M_LUNG_52__CCATTCGTCGCTGATA,CCATTCGTCGCTGATA-1-31-0-0-mouse-hlm,...,MACA_18m_M_LUNG_52,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary,endothelial: mouse,endothelial: Capillary: mouse,mouse_18-M-52


### Merge signatures on celltypes

In [26]:


# grouped = train_sig_df_celltypes.groupby(
#     ["sketch_id", "alphabet", "ksize", "broad_group"]
# )
# print(len(grouped))





# per_celltype_sigs = Parallel(n_jobs=96)(
#     delayed(sig_utils.merge_celltype_sigs)(
#         sketch_id,
#         ksize,
#         moltype,
#         broad_group,
#         df,
#         merged_celltype_outdir_base=train_2_merged_celltype,
#         dryrun=False,
#         force=True,
        
#     )
#     for (sketch_id, moltype, ksize, broad_group), df in tqdm(
#         grouped, total=len(grouped)
#     )
# )


# per_celltype_sigs_df = pd.Series(per_celltype_sigs, index=grouped.groups).reset_index()
# per_celltype_sigs_df = per_celltype_sigs_df.rename(
#     columns={
#         "level_0": "sketch_id",
#         "level_1": "alphabet",
#         "level_2": "ksize",
#         "level_3": "broad_group",
#         0: "sig_path",
#     }
# )
# describe(per_celltype_sigs_df)

100%|██████████| 30/30 [00:00<00:00, 894.70it/s]

30


(30, 5)
--- First 5 entries ---


,sketch_id,alphabet,ksize,broad_group,sig_path
0,alphabet-DNA__ksize-21__scaled-10,DNA,21,Alveolar Epithelial Type 2,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...
1,alphabet-DNA__ksize-21__scaled-10,DNA,21,B cell,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...
2,alphabet-DNA__ksize-21__scaled-10,DNA,21,Capillary,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...
3,alphabet-DNA__ksize-21__scaled-10,DNA,21,Dendritic,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...
4,alphabet-DNA__ksize-21__scaled-10,DNA,21,Fibroblast,/mnt/ibm_sm/olga/immune-evolution/kmer-signatu...


In [116]:
thresholds = (0.05, 0.1)

grouped = train_sig_df_celltypes.groupby(
    ["sketch_id", "alphabet", "ksize", "broad_group"]
)
print(len(grouped))


for threshold in thresholds:
    threshold_outdir = f'{train_2_merged_celltype}--min-kmer-count--{int(threshold*100):d}-percent'
#     if os.path.exists(threshold_outdir):
#         continue
    ! mkdir $threshold_outdir
    print(threshold_outdir)
    Parallel(n_jobs=96)(
        delayed(sig_utils.merge_celltype_sigs)(
            sketch_id,
            ksize,
            moltype,
            broad_group,
            df,
            merged_celltype_outdir_base=threshold_outdir,
            dryrun=False,
            force=True,
            min_sig_fraction=threshold,
        )
        for (sketch_id, moltype, ksize, broad_group), df in tqdm(
            grouped, total=len(grouped)
        )
    )
    ! sourmash sig describe \
        --csv $threshold_outdir/sourmash_sig_describe.csv \
        $threshold_outdir/*/*.sig

30
mkdir: cannot create directory ‘/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/2--merged-celltype-sigs--min-kmer-count--5-percent’: File exists



100%|██████████| 30/30 [00:00<00:00, 782.25it/s]

/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/2--merged-celltype-sigs--min-kmer-count--5-percent



== This is sourmash version 4.0.0. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

---<<<ercent/alphabet-dayhoff__ksize-51__scaled-10/alveolar_epithelial_type_2.sig'
signature filename: /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/2--merged-celltype-sigs--min-kmer-count--5-percent/alphabet-dayhoff__ksize-51__scaled-10/alveolar_epithelial_type_2.sig
signature: Alveolar Epithelial Type 2
source file: ** no name **
md5: d95690dab98d20467fd8b5525a7fcfb3
k=17 molecule=dayhoff num=0 scaled=10 seed=42 track_abundance=1
size: 67496
signature license: CC0

<<<ercent/alphabet-dayhoff__ksize-51__scaled-10/alveolar_epithelial_type_2.sig'
---<<<-min-kmer-count--5-percent/alphabet-dayhoff__ksize-51__scaled-10/b_cell.sig'
signature filename: /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/2--merged-celltype-sigs--min-kmer-count--5-percent/alphabet-da


  0%|          | 0/30 [00:00<?, ?it/s]

/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/2--merged-celltype-sigs--min-kmer-count--10-percent


100%|██████████| 30/30 [00:00<00:00, 886.30it/s]



== This is sourmash version 4.0.0. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

---<<<ercent/alphabet-dayhoff__ksize-51__scaled-10/alveolar_epithelial_type_2.sig'
signature filename: /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/2--merged-celltype-sigs--min-kmer-count--10-percent/alphabet-dayhoff__ksize-51__scaled-10/alveolar_epithelial_type_2.sig
signature: Alveolar Epithelial Type 2
source file: ** no name **
md5: b4aa9df7dbdd8ba40f3ad8a71f82d828
k=17 molecule=dayhoff num=0 scaled=10 seed=42 track_abundance=1
size: 14843
signature license: CC0

<<<ercent/alphabet-dayhoff__ksize-51__scaled-10/alveolar_epithelial_type_2.sig'
---<<<min-kmer-count--10-percent/alphabet-dayhoff__ksize-51__scaled-10/b_cell.sig'
signature filename: /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/2--merged-celltype-sigs--min-kmer-count--10-percent/alphabet-

### REmove hashes present across 80% of cell types, after keeping only k-mers shared by 5%, 10% of cells within a celltypes

In [117]:
sketch_id_to_broad_group = train_sig_df_celltypes.groupby(
    ["sketch_id", "alphabet", "ksize", 'broad_group'], observed=True
).size()
sketch_id_to_broad_group.name = 'n_cells'
sketch_id_to_broad_group = sketch_id_to_broad_group.reset_index()
sketch_id_to_broad_group['sanitized'] = sketch_id_to_broad_group.broad_group.map(sig_utils.sanitize)
describe(sketch_id_to_broad_group)


(30, 6)
--- First 5 entries ---


,sketch_id,alphabet,ksize,broad_group,n_cells,sanitized
0,alphabet-DNA__ksize-21__scaled-10,DNA,21,Monocyte,533,monocyte
1,alphabet-DNA__ksize-21__scaled-10,DNA,21,T cell,104,t_cell
2,alphabet-DNA__ksize-21__scaled-10,DNA,21,Alveolar Epithelial Type 2,19,alveolar_epithelial_type_2
3,alphabet-DNA__ksize-21__scaled-10,DNA,21,Capillary,280,capillary
4,alphabet-DNA__ksize-21__scaled-10,DNA,21,Fibroblast,51,fibroblast


In [118]:
thresholds = 0.05, 0.1

for threshold in thresholds:
    threshold_indir_base = (
         f'{train_2_merged_celltype}--min-kmer-count--{int(threshold*100):d}-percent'
    )
    threshold_outdir = f"{train_3_merged_celltype_remove_common}--min-kmer-count--{int(threshold*100):d}-percent"

    sketch_id_to_broad_group["sig_path"] = sketch_id_to_broad_group.apply(
        lambda x: os.path.join(
            threshold_indir_base, x["sketch_id"], x["sanitized"] + ".sig"
        ),
        axis=1,
    )
    describe(sketch_id_to_broad_group)
    grouped = sketch_id_to_broad_group.groupby(['sketch_id', 'alphabet', 'ksize'])
    print(threshold_outdir)
    sigs_without_common_hashes = Parallel(n_jobs=64)(
        delayed(sig_utils.remove_common_hashes_from_sig_df)(
            df,
            sketch_id,
            ksize,
            moltype,
            fraction_threshold=0.8,
            output_dir=threshold_outdir,
            force=True,
            #         create_hash_count_csv=False,
        )
        for (sketch_id, moltype, ksize), df in tqdm(grouped, total=len(grouped))
    )
    ! sourmash sig describe \
        --csv $threshold_outdir/sourmash_sig_describe.csv \
        $threshold_outdir/*/*.sig

(30, 7)
--- First 5 entries ---


,sketch_id,alphabet,ksize,broad_group,n_cells,sanitized,sig_path
0,alphabet-DNA__ksize-21__scaled-10,DNA,21,Monocyte,533,monocyte,/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signa...
1,alphabet-DNA__ksize-21__scaled-10,DNA,21,T cell,104,t_cell,/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signa...
2,alphabet-DNA__ksize-21__scaled-10,DNA,21,Alveolar Epithelial Type 2,19,alveolar_epithelial_type_2,/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signa...
3,alphabet-DNA__ksize-21__scaled-10,DNA,21,Capillary,280,capillary,/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signa...
4,alphabet-DNA__ksize-21__scaled-10,DNA,21,Fibroblast,51,fibroblast,/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signa...



  0%|          | 0/3 [00:00<?, ?it/s]

/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/3--merged-celltype-remove-common-kmers--min-kmer-count--5-percent



100%|██████████| 3/3 [00:02<00:00,  1.48it/s]


...

,sketch_id,alphabet,ksize,broad_group,n_cells,sanitized,sig_path
0,alphabet-DNA__ksize-21__scaled-10,DNA,21,Monocyte,533,monocyte,/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signa...
1,alphabet-DNA__ksize-21__scaled-10,DNA,21,T cell,104,t_cell,/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signa...
2,alphabet-DNA__ksize-21__scaled-10,DNA,21,Alveolar Epithelial Type 2,19,alveolar_epithelial_type_2,/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signa...
3,alphabet-DNA__ksize-21__scaled-10,DNA,21,Capillary,280,capillary,/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signa...
4,alphabet-DNA__ksize-21__scaled-10,DNA,21,Fibroblast,51,fibroblast,/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signa...



  0%|          | 0/3 [00:00<?, ?it/s]

/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/3--merged-celltype-remove-common-kmers--min-kmer-count--10-percent


100%|██████████| 3/3 [00:00<00:00, 447.25it/s]



== This is sourmash version 4.0.0. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

---<<<ercent/alphabet-dayhoff__ksize-51__scaled-10/alveolar_epithelial_type_2.sig'
signature filename: /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/3--merged-celltype-remove-common-kmers--min-kmer-count--10-percent/alphabet-dayhoff__ksize-51__scaled-10/alveolar_epithelial_type_2.sig
signature: Alveolar Epithelial Type 2
source file: ** no name **
md5: 0ae3ef21787d1fc7d3521552ce3e4850
k=17 molecule=dayhoff num=0 scaled=10 seed=42 track_abundance=1
size: 12945
signature license: CC0

<<<ercent/alphabet-dayhoff__ksize-51__scaled-10/alveolar_epithelial_type_2.sig'
---<<<min-kmer-count--10-percent/alphabet-dayhoff__ksize-51__scaled-10/b_cell.sig'
signature filename: /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/3--merged-celltype-remove-common-kmers--min-km

In [113]:
ls -lha $threshold_indir_base

total 1.0K
drwxr-xr-x  2 olga czb 4.0K Mar 27 13:28 ./
drwxr-xr-x 10 olga czb 4.0K Mar 27 13:31 ../


In [112]:
ls -lha $threshold_outdir/*


/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/3--merged-celltype-remove-common-kmers--min-kmer-count--5-percent/alphabet-dayhoff__ksize-51__scaled-10:
total 1.0K
drwxr-xr-x 2 olga czb 4.0K Mar 27 13:31 ./
drwxr-xr-x 5 olga czb 4.0K Mar 27 13:31 ../

/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/3--merged-celltype-remove-common-kmers--min-kmer-count--5-percent/alphabet-DNA__ksize-21__scaled-10:
total 1.0K
drwxr-xr-x 2 olga czb 4.0K Mar 27 13:31 ./
drwxr-xr-x 5 olga czb 4.0K Mar 27 13:31 ../

/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/3--merged-celltype-remove-common-kmers--min-kmer-count--5-percent/alphabet-protein__ksize-30__scaled-10:
total 1.0K
drwxr-xr-x 2 olga czb 4.0K Mar 27 13:31 ./
drwxr-xr-x 5 olga czb 4.0K Mar 27 13:31 ../


### Remove hashes present across 80% of celltypes

In [119]:
# %%time

# grouped = per_celltype_sigs_df.groupby(
#     ["sketch_id", "alphabet", "ksize"]
# )


# sigs_without_common_hashes = Parallel(n_jobs=64)(
#     delayed(sig_utils.remove_common_hashes_from_sig_df)(
#         df,
#         sketch_id,
#         ksize,
#         moltype,
#         fraction_threshold=0.8,
#         output_dir=train_3_merged_celltype_remove_common,
#         force=True,
#         create_hash_count_csv=True,
#     )
#     for (sketch_id, moltype, ksize), df in tqdm(grouped, total=len(grouped))
# )
# len(sigs_without_common_hashes)

In [120]:
# !rm -rf $train_3_merged_celltype_remove_common/dayhoff
# !rm -rf $train_3_merged_celltype_remove_common/DNA
# !rm -rf $train_3_merged_celltype_remove_common/protein

In [121]:
# ls -lha $train_3_merged_celltype_remove_common/alphabet-dayhoff__ksize-51__scaled-10/

## Make celltype databases

In [56]:
index_txt = os.path.join(train_4_celltype_sbts, 'sourmash_index_commands.txt')


with open(index_txt, 'w') as f:
    for input_dir in glob.glob(os.path.join(train_3_merged_celltype_remove_common, "*")):
        sketch_id = os.path.basename(input_dir)
        print(sketch_id)
        try:
            alpha_moltype, alphabet, ksize, style, value = re.findall(
                sig_utils.SKETCH_INFO_PATTERN, sketch_id
            )[0]
        except IndexError:
            # Not a real sketch id --> skip
            continue
        sbt_zip = os.path.join(train_4_celltype_sbts, sketch_id + ".sbt.zip")
        command = (
            f"sourmash index --ksize {ksize} --{alphabet.lower()} "
            f"--traverse-directory {sbt_zip} {input_dir}"
        )
        f.write(command + '\n')
print(f'time parallel --progress --eta --jobs 10 --eta < {index_txt}')

alphabet-dayhoff__ksize-51__scaled-10
alphabet-DNA__ksize-21__scaled-10
alphabet-protein__ksize-30__scaled-10
time parallel --progress --eta --jobs 10 --eta < /mnt/ibm_sm/olga/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/4--celltype-sbt-dbts/sourmash_index_commands.txt


In [61]:
ls -lha $train_4_celltype_sbts

total 64M
drwxr-xr-x 2 olga czb 4.0K Feb 28 16:37 ./
drwxr-xr-x 7 olga czb 4.0K Feb 28 09:53 ../
-rw-r--r-- 1 olga czb  13M Feb 28 16:37 alphabet-dayhoff__ksize-51__scaled-10.sbt.zip
-rw-r--r-- 1 olga czb  37M Feb 28 16:37 alphabet-DNA__ksize-21__scaled-10.sbt.zip
-rw-r--r-- 1 olga czb  15M Feb 28 16:37 alphabet-protein__ksize-30__scaled-10.sbt.zip
-rw-r--r-- 1 olga czb 1.2K Feb 28 16:37 sourmash_index_commands.txt


In [43]:
alphabet

'protein'

In [30]:
re.findall(sig_utils.SKETCH_INFO_PATTERN, sketch_id)

[('alphabet', 'dayhoff', '51', 'scaled', '10')]

# Test data

## Copy mouse2 fastas

In [104]:
mouse2_obs = adata_shared.obs.query('channel == "MACA_18m_F_LUNG_51"')

mouse_fasta_input_dir = os.path.join(mouse_kmermaid_base, "translate")

sig_reorg.copy_fastas(
    test_0_fastas,
    mouse_fasta_input_dir,
    select_cell_ids=mouse2_obs.index,
    dryrun=False,
)

0it [00:00, ?it/s]

Copying peptides


27200it [01:23, 325.39it/s] 
0it [00:00, ?it/s]

Copying nucleotides


27200it [01:47, 252.18it/s] 


## Copy mouse2 sketches

In [57]:
mouse2_obs = adata_shared.obs.query('channel == "MACA_18m_F_LUNG_51"')


sig_reorg.copy_nucleotide_peptide_sketches(
    peptide_sketch_dir=mouse_peptide_sketch_dir,
    nucleotide_sketch_dir=mouse_nucleotide_sketch_dir,
    dryrun=False,
    pre_sketch_id_outdir=test_1_single_cells,
    select_cell_ids=mouse2_obs.index
)

5it [00:00, 41.73it/s]

Copying alphabet-DNA__ksize-21__scaled-10


7109it [00:39, 182.06it/s]
19it [00:00, 162.59it/s]

Copying alphabet-protein__ksize-30__scaled-10


7109it [00:16, 418.65it/s]
57it [00:00, 544.11it/s]

Copying alphabet-dayhoff__ksize-51__scaled-10


7109it [00:11, 623.93it/s]


In [60]:
ls -lha $test_1_single_cells

total 769K
drwxr-xr-x 5 olga czb 4.0K Feb 28 10:48 ./
drwxr-xr-x 4 olga czb 4.0K Feb 28 09:53 ../
drwxr-xr-x 2 olga czb 256K Feb 28 16:39 alphabet-dayhoff__ksize-51__scaled-10/
drwxr-xr-x 2 olga czb 256K Feb 28 16:39 alphabet-DNA__ksize-21__scaled-10/
drwxr-xr-x 2 olga czb 256K Feb 28 16:39 alphabet-protein__ksize-30__scaled-10/


# Create queries

In [101]:
test_dir

'/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/1--test-mouse2'

In [102]:
mouse2mouse_dir = '/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse'

In [1]:
test_1_single_cells

NameError: name 'test_1_single_cells' is not defined

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [86]:
moltypes = 'DNA', 'protein', 'dayhoff'

alpha_ksizes = (("DNA", 21), ("protein", 30), ("dayhoff", 51))


# Try both with removed common hashes and original
celltype_dirs = dict(
    remove_common=train_3_merged_celltype_remove_common,
    raw_merged=train_2_merged_celltype,
)

txts = []

for i, (name, celltype_dir) in enumerate(celltype_dirs.items()):
    this_celltype_dir_results = os.path.join(
        mouse2mouse_dir, f"3-{i}--search-results--{name}"
    )
    if not os.path.exists(this_celltype_dir_results):
        os.makedirs(this_celltype_dir_results)
    for alpha, ksize in alpha_ksizes:
        txt = csc_utils.make_sourmash_search_commands(
            this_celltype_dir_results, 
            query_sigs_dir=test_1_single_cells, 
            sbt_base_dir=train_4_celltype_sbts, 
            k_sizes=[ksize], 
            scaled_sizes=[10,],
            cell_ids=[],
            sbt_template_basename=None,
            query_sig_files=False, 
            containment=False,
            moltype=alpha,
            threshold=1e-10,
            n_jobs=96,
            force=True,
            traverse_directory=True,
        )
        txts.append(txt)
txts_to_cat = ' '.join(txts)
catted_txt = os.path.join(results_dir, 'sourmash_search_commands_concatenated.txt')
! cat $txts_to_cat > $catted_txt

print(f'---\nparallel --progress --eta --jobs 96 < {catted_txt}')

parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/1--test-mouse2/3-0--search-results--remove_common/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/1--test-mouse2/3-0--search-results--remove_common/sourmash_search_commands_protein.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/1--test-mouse2/3-0--search-results--remove_common/sourmash_search_commands_dayhoff.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/1--test-mouse2/3-1--search-results--raw_merged/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mous

## Make queries with circumstantial (present in only a few cells/celltype) and unspecific (present in many cell types) k-mers removed

In [126]:
moltypes = 'DNA', 'protein', 'dayhoff'

alpha_ksizes = (("DNA", 21), ("protein", 30), ("dayhoff", 51))

train_3_sigs_min_5_percent = os.path.join(train_dir, '2--merged-celltype-sigs--min-kmer-count--5-percent')
train_3_sigs_min_10_percent = os.path.join(train_dir, '2--merged-celltype-sigs--min-kmer-count--10-percent')
train_3_sigs_min_5_percent_remove_common = os.path.join(train_dir, '3--merged-celltype-remove-common-kmers--min-kmer-count--5-percent')
train_3_sigs_min_10_percent_remove_common = os.path.join(train_dir, '3--merged-celltype-remove-common-kmers--min-kmer-count--10-percent')


# Try both with removed common hashes and original
celltype_dirs = {
    'min-5-percent-shared-kmers': train_3_sigs_min_5_percent,
    'min-10-percent-shared-kmers': train_3_sigs_min_10_percent,
    'min-5-percent-shared-kmers-remove-common-kmers': train_3_sigs_min_5_percent_remove_common,
    'min-10-percent-shared-kmers-remove-common-kmers': train_3_sigs_min_10_percent_remove_common,
}
txts = []

for i, (name, celltype_dir) in enumerate(celltype_dirs.items()):
    this_celltype_dir_results = os.path.join(
        mouse2mouse_dir, f"3-{i}--search-results--{name}"
    )
    if not os.path.exists(this_celltype_dir_results):
        os.makedirs(this_celltype_dir_results)
    for alpha, ksize in alpha_ksizes:
        txt = csc_utils.make_sourmash_search_commands(
            this_celltype_dir_results, 
            query_sigs_dir=test_1_single_cells, 
            sbt_base_dir=celltype_dir, 
            k_sizes=[ksize], 
            scaled_sizes=[10,],
            cell_ids=[],
            sbt_template_basename=None,
            query_sig_files=False, 
            containment=False,
            moltype=alpha,
            threshold=1e-10,
            n_jobs=96,
            force=True,
            traverse_directory=True,
        )
        txts.append(txt)
txts_to_cat = ' '.join(txts)
catted_txt = os.path.join(results_dir, 'sourmash_search_commands_concatenated.txt')
! cat $txts_to_cat > $catted_txt

print(f'---\nparallel --progress --eta --jobs 96 < {catted_txt}')

parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/3-0--search-results--min-5-percent-shared-kmers/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/3-0--search-results--min-5-percent-shared-kmers/sourmash_search_commands_protein.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/3-0--search-results--min-5-percent-shared-kmers/sourmash_search_commands_dayhoff.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/3-1--search-results--min-10-percent-shared-kmers/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/3-

## CONTAINMENT - Make queries with circumstantial (present in only a few cells/celltype) and unspecific (present in many cell types) k-mers removed

In [127]:
moltypes = 'DNA', 'protein', 'dayhoff'

alpha_ksizes = (("DNA", 21), ("protein", 30), ("dayhoff", 51))

train_3_sigs_min_5_percent = os.path.join(train_dir, '2--merged-celltype-sigs--min-kmer-count--5-percent')
train_3_sigs_min_10_percent = os.path.join(train_dir, '2--merged-celltype-sigs--min-kmer-count--10-percent')
train_3_sigs_min_5_percent_remove_common = os.path.join(train_dir, '3--merged-celltype-remove-common-kmers--min-kmer-count--5-percent')
train_3_sigs_min_10_percent_remove_common = os.path.join(train_dir, '3--merged-celltype-remove-common-kmers--min-kmer-count--10-percent')


# Try both with removed common hashes and original
celltype_dirs = {
    'min-5-percent-shared-kmers': train_3_sigs_min_5_percent,
    'min-10-percent-shared-kmers': train_3_sigs_min_10_percent,
    'min-5-percent-shared-kmers-remove-common-kmers': train_3_sigs_min_5_percent_remove_common,
    'min-10-percent-shared-kmers-remove-common-kmers': train_3_sigs_min_10_percent_remove_common,
}
txts = []

for i, (name, celltype_dir) in enumerate(celltype_dirs.items()):
    this_celltype_dir_results = os.path.join(
        mouse2mouse_dir, f"3-{i}--containment-search-results--{name}"
    )
    if not os.path.exists(this_celltype_dir_results):
        os.makedirs(this_celltype_dir_results)
    for alpha, ksize in alpha_ksizes:
        txt = csc_utils.make_sourmash_search_commands(
            this_celltype_dir_results, 
            query_sigs_dir=test_1_single_cells, 
            sbt_base_dir=celltype_dir, 
            k_sizes=[ksize], 
            scaled_sizes=[10,],
            cell_ids=[],
            sbt_template_basename=None,
            query_sig_files=False, 
            containment=True,
            moltype=alpha,
            threshold=1e-10,
            n_jobs=96,
            force=True,
            traverse_directory=True,
        )
        txts.append(txt)
txts_to_cat = ' '.join(txts)
catted_txt = os.path.join(results_dir, 'sourmash_search_commands_concatenated.txt')
! cat $txts_to_cat > $catted_txt

print(f'---\nparallel --progress --eta --jobs 96 < {catted_txt}')

parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/3-0--containment-search-results--min-5-percent-shared-kmers/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/3-0--containment-search-results--min-5-percent-shared-kmers/sourmash_search_commands_protein.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/3-0--containment-search-results--min-5-percent-shared-kmers/sourmash_search_commands_dayhoff.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/3-1--containment-search-results--min-10-percent-shared-kmers/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_lg/data_sm_copy/immune-evolution/

In [93]:
ls -lha /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/1--test-mouse2/3-0--search-results--remove_common/alphabet-DNA__ksize-21__scaled-10/MACA_18m_F_LUNG_51__AAACGGGCAGTTCATG.csv

-rw-r--r-- 1 olga czb 749 Mar  3 14:18 /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/1--test-mouse2/3-0--search-results--remove_common/alphabet-DNA__ksize-21__scaled-10/MACA_18m_F_LUNG_51__AAACGGGCAGTTCATG.csv


In [100]:
! wc -l $catted_txt

13188 /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/2--search-results/sourmash_search_commands_concatenated.txt


In [87]:
! head -n 3 $catted_txt 

sourmash search --quiet --dna --no-protein --no-dayhoff --num-results 3 --threshold 1e-10 -k 21 --traverse-directory --output /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/1--test-mouse2/3-0--search-results--remove_common/alphabet-DNA__ksize-21__scaled-10/MACA_18m_F_LUNG_51__GTATTCTTCCTGCAGG.csv /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/1--test-mouse2/1--single-cell-sigs/alphabet-DNA__ksize-21__scaled-10/MACA_18m_F_LUNG_51__GTATTCTTCCTGCAGG.sig /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/0--train-mouse1/4--celltype-sbt-dbts/alphabet-DNA__ksize-21__scaled-10.sbt.zip
sourmash search --quiet --dna --no-protein --no-dayhoff --num-results 3 --threshold 1e-10 -k 21 --traverse-directory --output /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/1--test-mouse2/3-0--search-results--remove_common/alph

In [83]:
!hostname

lrrr


In [95]:
ls -lha /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/

total 2.5K
drwxr-xr-x 5 olga czb 4.0K Feb 28 16:45 ./
drwxr-xr-x 4 olga czb 4.0K Feb 28 09:53 ../
drwxr-xr-x 7 olga czb 4.0K Feb 28 09:53 0--train-mouse1/
drwxr-xr-x 6 olga czb 4.0K Feb 28 17:26 1--test-mouse2/
drwxr-xr-x 5 olga czb 4.0K Feb 28 16:48 2--search-results/


In [96]:
ls -lha /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/0--mouse2mouse/1--mouse2mouse/2--search-results/

total 14M
drwxr-xr-x 5 olga czb 4.0K Feb 28 16:48 ./
drwxr-xr-x 5 olga czb 4.0K Feb 28 16:45 ../
drwxr-xr-x 2 olga czb 256K Feb 28 17:51 alphabet-dayhoff__ksize-51__scaled-10/
drwxr-xr-x 2 olga czb 256K Feb 28 17:42 alphabet-DNA__ksize-21__scaled-10/
drwxr-xr-x 2 olga czb 256K Feb 28 17:47 alphabet-protein__ksize-30__scaled-10/
-rw-r--r-- 1 olga czb 8.7M Feb 28 20:35 sourmash_search_commands_concatenated.txt
-rw-r--r-- 1 olga czb 1.4M Feb 28 17:26 sourmash_search_commands_dayhoff.txt
-rw-r--r-- 1 olga czb 1.5M Feb 28 17:26 sourmash_search_commands_DNA.txt
-rw-r--r-- 1 olga czb 1.4M Feb 28 17:26 sourmash_search_commands_protein.txt
